# RESEARCH NOTEBOOK --> XGridT


In [2]:
# This is necessary to recognize the modules
import os
import sys
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [3]:
import pandas as pd
import pandas_ta as ta  # noqa: F401
from core.data_sources import CLOBDataSource

# Initialize the data source
clob = CLOBDataSource()

In [4]:
# Define the parameters
exchange = "binance_perpetual"
trading_pair = "1000PEPE-USDT"
interval = "3m"
days = 10

In [5]:
candles = await clob.get_candles_last_days(connector_name=exchange, trading_pair=trading_pair, interval=interval, days=days)

In [6]:
from core.features.candles.peak_analyzer import PeakAnalyzer
import plotly.graph_objects as go
from plotly.subplots import make_subplots

ema_short = 12
ema_medium = 47
ema_long = 100


df = candles.data.copy()
df.ta.ema(length=ema_short, append=True)
df.ta.ema(length=ema_medium, append=True)
df.ta.ema(length=ema_long, append=True)

short_ema = df[f"EMA_{ema_short}"]
medium_ema = df[f"EMA_{ema_medium}"]
long_ema = df[f"EMA_{ema_long}"]
close = df["close"]


long_condition = (short_ema > medium_ema) & (medium_ema > long_ema) & (short_ema > long_ema)
short_condition = (short_ema < medium_ema) & (medium_ema < long_ema) & (short_ema < long_ema)

df.loc[long_condition, "signal"] = 1
df.loc[short_condition, "signal"] = -1


fig = candles.fig(width=1200)
ema_fast = f'EMA_{ema_short}'
ema_med = f'EMA_{ema_medium}' 
ema_slow = f'EMA_{ema_long}'

fig.add_trace(go.Scatter(x=df.index, y=df[ema_fast],
                         line=dict(color='white', width=2),
                         name='Fast EMA'))
fig.add_trace(go.Scatter(x=df.index, y=df[ema_med],
                         line=dict(color='blue', width=2), 
                         name='Medium EMA'))
fig.add_trace(go.Scatter(x=df.index, y=df[ema_slow],
                         line=dict(color='violet', width=2),
                         name='Slow EMA'))

fig.show()


In [7]:
df

,timestamp,open,high,low,close,volume,quote_asset_volume,n_trades,taker_buy_base_volume,taker_buy_quote_volume,EMA_12,EMA_47,EMA_100,signal
timestamp,,,,,,,,,,,,,,
2025-08-21 16:00:00,1755792000,0.0102385,0.0102585,0.0102362,0.0102534,138252396,1416374.3390611,5460,63031443,645774.0527322,NaN,NaN,NaN,NaN
2025-08-21 16:03:00,1755792180,0.0102534,0.0102544,0.0102251,0.0102401,114794071,1175124.600444,4412,43557423,445693.0379619,NaN,NaN,NaN,NaN
2025-08-21 16:06:00,1755792360,0.0102401,0.01026,0.0102072,0.0102571,218492504,2234783.7087103,6386,84359471,862828.9057112,NaN,NaN,NaN,NaN
2025-08-21 16:09:00,1755792540,0.0102571,0.0102756,0.0102544,0.0102724,121455161,1246964.1385993,3846,71010033,729068.7616095,NaN,NaN,NaN,NaN
2025-08-21 16:12:00,1755792720,0.0102726,0.0102929,0.0102707,0.0102889,108290258,1113506.9126556,3256,68949934,708998.1719798,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-31 15:42:00,1756654920,0.0098345,0.0098536,0.0098285,0.00985,703164722,6920701.2115025,5834,385499563,3794768.8878524,0.00981905,0.00981656,0.00984059,NaN
2025-08-31 15:45:00,1756655100,0.0098494,0.0098563,0.0098421,0.0098546,117480219,1157096.6561428,2825,54990548,541631.9001022,0.00982452,0.00981814,0.00984087,NaN
2025-08-31 15:48:00,1756655280,0.0098546,0.0098593,0.0098406,0.0098555,78635737,774575.9688589,2246,37636955,370730.8320379,0.00982928,0.0098197,0.00984116,NaN


In [11]:
# Generate signal


short_ema = df[f"EMA_{ema_short}"]
medium_ema = df[f"EMA_{ema_medium}"]
long_ema = df[f"EMA_{ema_long}"]
close = df["close"]


long_condition = (short_ema > medium_ema) & (medium_ema > long_ema) & (short_ema > long_ema)
short_condition = (short_ema < medium_ema) & (medium_ema < long_ema) & (short_ema < long_ema)

df["signal"] = 0
df.loc[long_condition, "signal"] = 1
df.loc[short_condition, "signal"] = -1

In [12]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02,
                    subplot_titles=('OHLC with BB', 'MACD', 'Signal'),
                    row_heights=[0.6, 0.2, 0.2])

# Add candlestick
fig.add_trace(go.Candlestick(x=df.index,
                             open=df['open'],
                             high=df['high'],
                             low=df['low'],
                             close=df['close'],
                             name='Candlesticks'),
              row=1, col=1)


# Add EMAs
ema_fast = f'EMA_{ema_short}'
ema_med = f'EMA_{ema_medium}' 
ema_slow = f'EMA_{ema_long}'

fig.add_trace(go.Scatter(x=df.index, y=df[ema_fast],
                         line=dict(color='#00FF00', width=2),
                         name='Fast EMA'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df[ema_med],
                         line=dict(color='#FFA500', width=2), 
                         name='Medium EMA'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df[ema_slow],
                         line=dict(color='#0000FF', width=2),
                         name='Slow EMA'), row=1, col=1)



# Add the signal line
fig.add_trace(go.Scatter(x=df.index, y=df['signal'],
                         mode='lines',
                         name='Signal',
                         line=dict(color="white")),
              row=3, col=1)

# Update layout for dark theme
fig.update_layout(
    title=f'{exchange} - {trading_pair} - {interval}',
    width=1500, height=1000,
    font=dict(color='#e1e1e1'),
    plot_bgcolor='#1e1e1e',
    paper_bgcolor='#1e1e1e',
    xaxis_rangeslider_visible=False,
    legend=dict(bgcolor='rgba(0,0,0,0)'),
    yaxis=dict(title='Price'),
    yaxis2=dict(title='MACD', showgrid=False),
    yaxis3=dict(title='Signal', showgrid=False),
    showlegend=False
)

# Update axes
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#323232', zeroline=False)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#323232', zeroline=False)

# Show the plot
fig.show()
